In [1]:
object ⟹{
    def elim[A, B](f: A => B)(a: A): B = 
        f(a)
}

type ⊥ = Nothing 

type ¬[A] = A => ⊥

object ¬{
    def elim[A, C](n: ¬[A])(a: A): C = n(a)
}

type ∨[A, B] = Either[A, B]

object ∨{
    def applyL[A, B](a: A): A ∨ B = Left(a)

    def applyR[A, B](b: B): A ∨ B = Right(b)
    
    def elim[A, B, C](f: A => C, g: B => C)(or: A ∨ B): C = 
        or.fold(f, g)
}

type ∧[A, B] = (A, B)

object ∧{
    def apply[A, B](a: A, b: B): A ∧ B = (a, b)
    
    def elimL[A, B](and: A ∧ B): A = and._1
    
    def elimR[A, B](and: A ∧ B): B = and._2
}



defined object ⟹
defined type ⊥
defined type ¬
defined object ¬
defined type ∨
defined object ∨
defined type ∧
defined object ∧

In [2]:
type <=>[A, B] = (A => B, B => A)
type <=[A, B] = B => A

defined type <=>
defined type <=

In [3]:
type Absurd = Nothing 

type Not[A] = A => Absurd

object Not{
    def elim[A, C](n: Not[A])(a: A): C = n(a)
}

type Or[A, B] = Either[A, B]

object Or{
    def applyL[A, B](a: A): Or[A, B] = Left(a)

    def applyR[A, B](b: B): Or[A, B] = Right(b)
    
    def elim[A, B, C](f: A => C, g: B => C)(or: Or[A, B]): C = 
        or.fold(f, g)
}

type And[A, B] = (A, B)

object And{
    def apply[A, B](a: A, b: B): And[A, B] = (a, b)
    
    def elimL[A, B](and: And[A, B]): A = and._1
    
    def elimR[A, B](and: And[A, B]): B = and._2
}

defined type Absurd
defined type Not
defined object Not
defined type Or
defined object Or
defined type And
defined object And

$\vdash X \wedge (X \Rightarrow Y) \Rightarrow Y$

In [4]:
def prop[X, Y]: (X, X => Y) => Y = 
    (x, f) => f(x)

defined function prop

In [5]:
def prop[X, Y]: ((X, X => Y)) => Y = 
    (a: (X, X => Y)) => 
        (a._2).apply(a._1)

defined function prop

In [6]:
def prop[X, Y]: X ∧ (X => Y) => Y = 
    (a: X ∧ (X => Y)) => 
        ⟹.elim(∧.elimR(a))(∧.elimL(a))

defined function prop

$X*(Y+Z) \cong X*Y + X*Z$

$\vdash (X \wedge (Y \vee Z)) \Leftrightarrow (X \wedge Y \vee X \wedge Z)$

In [7]:
def prop[P, Q, R]: (P ∧ (Q ∨ R)) <=> ((P ∧ Q) ∨ (P ∧ R)) = 
    ((a: P ∧ (Q ∨ R)) => 
        ∨.elim((q: Q) => ∨.applyL(∧.apply(∧.elimL(a), q)),
               (r: R) => ∨.applyR(∧.apply(∧.elimL(a), r)))(∧.elimR(a)),
     ∨.elim((a: P ∧ Q) => ∧.apply(∧.elimL(a), ∨.applyL(∧.elimR(a))),
            (a: P ∧ R) => ∧.apply(∧.elimL(a), ∨.applyR(∧.elimR(a)))))

defined function prop

In [8]:
def prop[P, Q, R]: (P, Either[Q, R]) <=> Either[(P, Q), (P, R)] = 
    ({ case (p, Left(q)) => Left((p,q))
        case (p, Right(r)) => Right((p,r)) },
    { case Left((p,q)) => (p, Left(q))
        case Right((p,r)) => (p, Right(r)) })

defined function prop

$X^{Y+Z} \cong X^Y * X^Z$

$\vdash (Y \vee Z \Rightarrow X) \Leftrightarrow (Y \Rightarrow X \wedge Z \Rightarrow X)$

In [9]:
def prop[X, Y, Z]: (Y ∨ Z => X) <=> (Y => X ∧ Z => X) = 
    (??? : (Y ∨ Z => X) => (Y => X ∧ Z => X), 
     ??? : (Y ∨ Z => X) <= (Y => X ∧ Z => X))

defined function prop

In [10]:
def prop[X, Y, Z]: (Either[Y, Z] => X) <=> (Y => X, Z => X) = 
    (f => (y => f(Left(y)), z => f(Right(z))) , 
     { case (f, g) => 
         { case Left(y) => f(y)
           case Right(z) => g(z)
         }
     })

defined function prop

$\vdash P \vee Q \Rightarrow \neg P \Rightarrow Q$ 

In [11]:
def proof12[P, Q]: Either[P, Q] => (P => Nothing) => Q = 
    (e: Either[P, Q]) => (f: P => Nothing) => 
        //e.fold[B](f, identity)
        e match {
            case Left(a: P) => f(a): Q
            case Right(b: Q) => b : Q
        }

def proof14[P, Q]: P ∨ Q => ¬[P] => Q = 
    (or: P ∨ Q) => (n: ¬[P]) => 
        ∨.elim(¬.elim[P, Q](n), identity[Q])(or)



defined function proof12
defined function proof14

In [12]:

def proof8[A, B]: Or[A, B] => Or[B, A] = 
    Or.elim[A, B, Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

// Not and implies

def proof3[A, B]: (A => Not[B]) => (A => B) => Not[A] = 
    i => (i2: A => B) => (a: A) => i(a)(i2(a))

def proof2[A, B]: Not[A] => A => B = 
    n => Not.elim[A, B](n)

def proof4[A, B]: Not[A => B] => Not[B] = 
    n => (b: B) => n((_: A) => b)

// Not and and

def proof5[A, B]: Not[A] => Not[And[A, B]] = 
//    (and: And[A, B]) => n(And.elimL(and))
    n => n compose And.elimL

def proof7[A, B]: Not[And[A, B]] => A => Not[B] = 
    n => (a: A) => (b: B) => n(And(a, b))

def proof11[A]: Not[A And Not[A]] = 
    a => And.elimR(a)(And.elimL(a))

// Not and or

def proof9[A, B]: Not[A Or B] => Not[A] And Not[B] = 
    n => And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

def proof10[A, B]: (Not[A] And Not[B]) => Not[A Or B] = 
    and => Or.elim[A, B, Absurd](And.elimL(and), And.elimR(and))



defined function proof8
defined function proof3
defined function proof2
defined function proof4
defined function proof5
defined function proof7
defined function proof11
defined function proof9
defined function proof10

In [13]:
trait ClassicalLogic{
    trait DN{
        def apply[A](nn: Not[Not[A]]): A
    }
    
    val dn: DN
    
    def proof1[A]: (Not[A] => A) => A =
        f => dn((n: Not[A]) => proof11(And(f(n),n)))
}


defined trait ClassicalLogic

All P are S
All S are R
All P are R

In [15]:
def proof[P, S, R]: ((P => S, S => R)) => P => R = 
    t => (p: P) => t._2(t._1(p))

defined function proof